In [18]:
import numpy as np
import torch
import botorch



In [24]:

problem = botorch.test_functions.Ackley(dim=2)

test = botorch.utils.draw_sobol_samples(
            bounds=problem.bounds, n=3, q=1).squeeze().tolist()

In [34]:
np.array(test[0]).tolist()

In [39]:
class Holder:
    def __init__(self, ref_to_dict):
        self._ref_to_dict = ref_to_dict

    def print(self):
        print(self._ref_to_dict)

class Main:
    def __init__(self):
        self._m_dict = {}
        self._several_holders = []
        for _ in range(5):
            self._several_holders.append(Holder(self._m_dict))

    def add(self, k, v):
        self._m_dict[k] = v

    def print(self):
        print("Main: ", self._m_dict)
        for i in range(5):
            print("Holder i: ", i)
            self._several_holders[i].print()

    def delete(self, k):
        del self._m_dict[k]
        print("After deleting:")
        self.print()

test = Main()
test.add("hi", "bo")
test.add("red", "2")
test.add("hi", "ooobo")

test.print()

test.delete("hi")

In [56]:
import networkx as nx
from IPython.core.display import SVG, display

def draw(graph, path = None):
    svg = nx.nx_agraph.to_agraph(graph).draw(path = path, prog = 'dot', format = 'svg')
    display(SVG(svg))

dag = nx.DiGraph()

dag.add_edges_from([('x1', 'z1'), ('x2', 'z1'), ('x2', 'z2'), ('z1', 'y'), ('z2', 'y')])
sub = dag.subgraph(['x1', 'y', 'z1'])

draw(sub)

print(list(nx.topological_sort(dag)))
print(list(nx.topological_sort(sub)))

In [ ]:
@dataclasses.dataclass(init=False, frozen=True)
class AckleySpace(ParamsSpace):
    x1: boxes.ContinuousBox = boxes.ContinuousBox(-32.768, 32.768)
    x2: boxes.ContinuousBox = boxes.ContinuousBox(-32.768, 32.768)
    x3: boxes.ContinuousBox = boxes.ContinuousBox(-32.768, 32.768)
    x4: boxes.ContinuousBox = boxes.ContinuousBox(-32.768, 32.768)
    x5: boxes.ContinuousBox = boxes.ContinuousBox(-32.768, 32.768)
    x6: boxes.ContinuousBox = boxes.ContinuousBox(-32.768, 32.768)



def eval_structure(X: torch.Tensor) -> Mapping[str, torch.Tensor]:
    part1 = torch.pow(X, 2)
    part2 = torch.cos(problem.c * X)

    return {"part1": part1, "part2": part2}
problem = botorch.test_functions.Ackley(negate=True, dim=6, noise_std=0.001)
space = AckleySpace()
dag = nx.DiGraph()
dag.add_node("y")
for i in space:
    dag.add_node(i)
    # One term
    # dag.add_edges_from([(i, f"z({i})"), (f"z({i})", "y")])
    # Both terms:
    # dag.add_edges_from([(i, f"f1({i})"), (i, f"f2({i})"),
    #                     (f"f1({i})", "y"), (f"f2({i})", "y")
    #                     ])
    dag.add_edge(i, 'y')